# 3. Create Required Azure Services
Before starting this section, make sure you have set up the global variables as shown in the previous section [02_setup_environment.ipynb](02_setup_environment.ipynb). Once your global variables have been set up, you no longer have to set them up again, as they are stored in a .env environment file. 

If you delete your .env file or your Azure login session expires, you may need to re-run the setup process. Keep this in mind when proceeding onto the other sections in this sample.

## 3.1. Get Global Variables
First, we will read the previously stored global variables.

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

azureSubsctiptionId = get_key(envPath, "azureSubsctiptionId")
resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
acrServiceName = get_key(envPath, "acrServiceName")
iotHubServiceName = get_key(envPath, "iotHubServiceName")
iotDeviceId = get_key(envPath, "iotDeviceId")
mediaServiceName = get_key(envPath, "mediaServiceName")
storageServiceName = get_key(envPath, "storageServiceName")

## 3.2. Create Resource Group
We will now create a new Azure resource group using the resource group name and location set in the previous section [02_setup_environment.ipynb](02_setup_environment.ipynb). Check that this name is present in the .env file, as this name will be auto-filled into the placeholders starting with the character "$". 

In [ ]:
!az group create --name $resourceGroupName --location  $resourceLocation

## 3.3. Create Azure Container Registry Service
We will use Azure Container Registry (ACR) to store our module images. Again, the placeholders starting with the character "$" will be auto-filled with the values we stored in the .env file.

In [ ]:
!az acr create --name $acrServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Basic --admin-enabled true

> <span style="color:red; font-weight: bold"> [!WARNING] </span>  
> Even if the cell above finishes executing, it may take several seconds to minutes to have the changes reflected on Azure Datacenter. Thus, wait a bit before proceeding to the next cell to avoid running into errors.

### 3.3.1. Get Access Credentials from ACR
After the ACR resource has been generated, we can grab the access credentials from ACR by running the following code snippets.

In [ ]:
%%bash --out output -s "$acrServiceName"
az acr credential show -n $1

In [ ]:
import json
jsonRegDetails = json.loads(output)

acrUserName = jsonRegDetails['username']
acrPassword = jsonRegDetails['passwords'][0]['value']

Save these ACR credentials for future use by storing them in the .env file.

In [ ]:
set_key(envPath, "acrUserName", acrUserName)
set_key(envPath, "acrPassword", acrPassword)
set_key(envPath, "acrServiceFullName", acrServiceName+".azurecr.io")

## 3.4. Create Azure IoT Hub
Next, we will create an Azure IoT Hub.
> <span style="color:red; font-weight: bold"> [!WARNING] </span>  
> Running the code snippet below may yield the following error: "IotHub name '<NAME\>' is not available". This error appears if the service already exists. If you see the error, skip the step, as you have likely created the resource already.


In [ ]:
!az iot hub create --name $iotHubServiceName --resource-group $resourceGroupName --location $resourceLocation --sku S1

Save these Azure IoT Hub connection string for future use by storing it in the .env file. This string is used to identify the hub.

In [ ]:
%%bash --out iotHubConnString -s "$resourceGroupName" "$iotHubServiceName"

az iot hub show-connection-string --hub-name $2 --output tsv

In [ ]:
set_key(envPath, "iotHubConnString", iotHubConnString.rstrip())

## 3.5. Create IoT Edge Device Identity
To run LVA on the Edge, we will need to create an IoT Edge device to run our modules with AI capability. In the code below, we will instruct Azure IoT Hub to create a resource framework for this cloud-based IoT Edge device.

In [ ]:
# Install iot hub CLI extensions
!az extension add --name azure-cli-iot-ext

If we want to connect/match physical devices to this cloud-based Edge device later, the connection will be set using the IoT Edge device's connection string. Everything in this cloud-based Edge device will be cloned into the physical device, including all the IoT Edge modules. Thus, we will also be printing out and saving the cloud-based IoT Edge device's connection string for future use.

In [ ]:
%%bash --out iotEdgeDeviceConnString -s "$resourceGroupName" "$iotHubServiceName" "$iotDeviceId"
# Create edge device if not already exists
az iot hub device-identity show --resource-group $1 --hub-name $2 --device-id $3 &> /dev/null

if [ $? -ne 0 ]; then
    az iot hub device-identity create --hub-name $2 --device-id $3 --edge-enabled &> /dev/null
fi

az iot hub device-identity show-connection-string --hub-name $2 --device-id $3 --output tsv

In [ ]:
set_key(envPath, "iotEdgeDeviceConnString", iotEdgeDeviceConnString.rstrip())

In [ ]:
# Print out IoT Edge Device deteails and store its connection string...
print('Azure IoT Hub Name: {}'.format(iotHubServiceName))
print('Azure IoT device id : {}'.format(iotDeviceId))
print('IoT Edge Device Connection String: {}'.format(iotEdgeDeviceConnString))
print('IoT Hub Connection String: {}'.format(iotHubConnString))

## 3.6. Create Azure Storage Services
To store assets onto Azure Media Services, we must also create a storage resource.

In [ ]:
!az storage account create --name $storageServiceName --resource-group $resourceGroupName --location $resourceLocation --sku Standard_LRS

## 3.7. Create Azure Media Services

Azure Media Services (AMS) is a cloud-based media workflow platform to index, package, protect, and stream video. For this sample, we will be using AMS to archive video clips (Edge streams) generated when our AI module detects motion.

In [ ]:
!az ams account create --name $mediaServiceName --resource-group $resourceGroupName --storage-account $storageServiceName --location $resourceLocation

## 3.8. Create Azure Service Principal
A service principal is a security identity used by user-created apps, services, and tools to access Azure resources. Like with the other resources created thus far, we will be creating a service principal and storing the credentials (AAD_TENANT_ID, AAD_SERVICE_PRINCIPAL_ID, and AAD_SERVICE_PRINCIPAL_SECRET) in the .env file. 


In [ ]:
%%bash --out output -s "$mediaServiceName" "$resourceGroupName" 

az ams account sp create --account-name $1 --resource-group $2 --name lvasp$2 --output json

if [ $? -ne 0 ]; then
    az ams account sp reset-credentials --account-name $1 --resource-group lvasp$2 --name $2 --output json
fi

In [ ]:
import json

amssp = json.loads(output)

aadTenantId = amssp['AadTenantId']
aadServicePrincipalId = amssp['AadClientId']
aadServicePrincipalSecret = amssp['AadSecret']

set_key(envPath, "aadTenantId", aadTenantId)
set_key(envPath, "aadServicePrincipalId", aadServicePrincipalId)
set_key(envPath, "aadServicePrincipalSecret", aadServicePrincipalSecret)